Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

### Описание данных  
#### Признаки  
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя
#### Целевой признак
* Price — цена (евро)

### План решения:
##### Часть 1. Подготовка данных:
* [Подготовка данных]((#1st_chap)) Загрузим датасет, проверим таблицу на наличие дубликатов, удалим колонки с данными, которые не понадобятся нам для обучения моделей, заполним пропущенные значения, разделим датасет на обучающую и тестовую выборки, создадим кросс-валидатор для последующего обучения моделей.  
* [Обучение моделей]((#2nd_chap)) Напишем функцию оценки моделей, принимающую на вход модель обучения, и данные, и возвращающую время обучения и предсказания RMSE, и RMSE на кросс-валидации, Обучим и измерим время выполнения кода и rmse для моделей Ridge , LightGBM , CatBoost и с помощью гридсёрча выберем наиболее подходящие параметры модели.  
* [Анализ результатов]((#3rd_chap)) Объединим в таблицу показатели времени обучения и качества наших моделей и выберем наиболее подходящую. 

<a id='1st_chap'></a>
# 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error as mse
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
import time
from sklearn.pipeline import Pipeline

In [2]:
cars.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,0,60437,2016-04-06 10:17:21


In [3]:
cars.shape

(354369, 16)

In [4]:
cars.duplicated().sum()

4

In [5]:
cars.drop_duplicates(inplace=True)

In [6]:
cars.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354365.000000,354365.000000,354365.000000,354365.000000,354365.000000,354365.0,354365.000000
mean,4416.679830,2004.234481,110.093816,128211.363989,5.714650,0.0,50508.503800
std,4514.176349,90.228466,189.851330,37905.083858,3.726432,0.0,25783.100078
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [7]:
cars.NumberOfPictures.value_counts()

0    354365
Name: NumberOfPictures, dtype: int64

In [8]:
# Удалим колонки не влияющие на обучение моделей:
# DateCrawled, DateCreated, LastSeen - даты скачивания и создания объявления, дата последней активности пользователя
# NumberOfPictures -  количество фотографий автомобиля во всех строчках значение 0
cars.drop(columns = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], inplace=True)

In [9]:
cars.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


In [10]:
# Выделим колонки, содержащие категориальные переменные
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [11]:
cars[cat_columns].isna().sum()

VehicleType    37490
Gearbox        19833
Model          19705
FuelType       32895
Brand              0
NotRepaired    71154
dtype: int64

In [12]:
# Заполним пропуски в категориальных данных текстом 'unknown'
cars[cat_columns] = cars[cat_columns].fillna('unknown')

In [13]:
# Разобъем датасет на обучающую и тестовую выборки и выделим целевой признак
X = cars.drop(columns=['Price'])
y = cars.Price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)


In [14]:
# создадим кросс-валидатор для последующего обучения моделей
cv = KFold(n_splits=3, shuffle=True, random_state=12345)

<a id='2nd_chap'></a>
# 2. Обучение моделей

#### Обучим и измерим  время выполнения кода и rmse для моделей Ridge , LightGBM , CatBoost 


In [15]:
# напишем функцию, принимающую модель и данные, и возвращающую время обучения и предсказания и RMSE
# Добавил возврат значения RMSE на кросс-валидации
def model_elector(model, features_train, target_train, features_test, target_test, best_score):
    now = time.time()
    model.fit(features_train, target_train)
    training_time = time.time() - now
    
    now = time.time()
    predicted = model.predict(features_test)
    predict_time = time.time() - now
    rmse = mse(target_test, predicted)**0.5
    cv_rmse = np.sqrt(-1 * best_score)
    return training_time, predict_time, rmse, cv_rmse

### LightGBM

In [16]:
# кодируем категориальные признаки через LabelEncoder и с помощью гридсёрча выберем наиболее подходящие параметры модели
encoder = LabelEncoder()
lgbm_train = X_train.copy()
lgbm_test = X_test.copy()

for col in cat_columns:
    lgbm_train[col] = encoder.fit_transform(lgbm_train[col])
    lgbm_test[col] = encoder.fit_transform(lgbm_test[col])
    
lgbm = LGBMRegressor(boosting_type='gbdt', random_state=12345)
params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'n_estimators': [40, 60],
    'num_leaves': [31, 41, 51],
}

gs_gbm =  GridSearchCV(lgbm,params, cv=cv, scoring='neg_mean_squared_error', verbose=True)

In [17]:
%%time
gs_gbm.fit(lgbm_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 10.9min finished


CPU times: user 10min 51s, sys: 4.17 s, total: 10min 55s
Wall time: 11min 2s


GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective...
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ]),
                  

In [18]:
gs_gbm.best_params_

{'learning_rate': 0.1778279410038923, 'n_estimators': 60, 'num_leaves': 51}

### CatBoost

In [19]:
cbr = CatBoostRegressor(random_seed=12345,
                        loss_function='RMSE',
                        silent=True,
                        cat_features=cat_columns)

params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'iterations': [40, 60],
    'depth': [d for d in range(2, 11)],
}

grid_cbr = GridSearchCV(cbr,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=False)

In [20]:
%%time
grid_cbr.fit(X_train, y_train);

CPU times: user 1h 9min 5s, sys: 5min 54s, total: 1h 14min 59s
Wall time: 1h 20min


GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7fb3b12f16d0>,
             iid='warn', n_jobs=None,
             param_grid={'depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'iterations': [40, 60],
                         'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=False)

In [21]:
grid_cbr.best_params_

{'depth': 10, 'iterations': 60, 'learning_rate': 0.1778279410038923}

### Ridge

In [22]:
# Подготовимм пайплайн one-hot encoder -> scaler -> Ridge
pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', Ridge(random_state=12345))
])

params = [
    {
        'model__alpha': np.logspace(-2, 3, 20)
    } 
]

In [23]:
grid_ridge = GridSearchCV(pipe,
                    param_grid=params,
                    cv=cv,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    verbose=False)

In [24]:
%%time
grid_ridge.fit(X_train, y_train);

CPU times: user 6min 52s, sys: 1.07 s, total: 6min 53s
Wall time: 6min 57s


GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('ohe',
                                        OneHotEncoder(categorical_features=None,
                                                      categories=None,
                                                      drop=None,
                                                      dtype=<class 'numpy.float64'>,
                                                      handle_unknown='ignore',
                                                      n_values=None,
                                                      sparse=True)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=False,
                                                       with_std=...
       1.12883789e

In [25]:
grid_ridge.best_params_

{'model__alpha': 1000.0}

In [26]:
ridge_rmse = np.sqrt(-1 *grid_ridge.best_score_)

<a id='3rd_chap'></a>
# 3. Анализ моделей

In [27]:
# Запишем данные всех моделей вместе
models = []
models.append(model_elector(grid_ridge.best_estimator_, X_train, y_train, X_test, y_test, grid_ridge.best_score_))
models.append(model_elector(grid_cbr.best_estimator_, X_train, y_train, X_test, y_test, grid_cbr.best_score_))
models.append(model_elector(gs_gbm.best_estimator_, lgbm_train, y_train, lgbm_test, y_test, gs_gbm.best_score_))

In [28]:
final_models = pd.DataFrame(data=models, index=['Ridge', 'CatBoost', 'LightGBM'], columns=['trainig_time', 'predic_time', 'RMSE', 'RMSE on CV'])

In [29]:
final_models

,trainig_time,predic_time,RMSE,RMSE on CV
Ridge,10.410765,0.167231,2142.078123,2142.506299
CatBoost,40.247549,0.188525,1820.532629,1825.716183
LightGBM,6.987270,0.505776,1860.705387,1820.655901


## Вывод:  
Наилучшие показатели скорости работы и RMSE на кросс-валидации у модели LightGBM.